In [1]:
from aiy.vision.inference import CameraInference
from aiy.vision.models import face_detection
from aiy.vision.streaming.server import StreamingServer
from aiy.vision.streaming import svg
from aiy.leds import Leds, Color
from gpiozero import Servo
from aiy.pins import PIN_A, PIN_B

from picamera import PiCamera
from IPython.display import Image, display, clear_output

import contextlib
import time

### Robot facing face

We are going to use the servo motor to point at the face that was detected. We will use the bounding box to determine the angle. And remember if your joy detector is running, you need to turn it off using the commands

```
sudo systemctl stop joy_detection_demo.service
```

First we  load our robot extenstion. THis is a version running a separate thread so the wait does not stop the face recognition.

In [2]:
from robot import RobotService

In [4]:
bot=RobotService()

/usr/lib/python3/dist-packages/gpiozero/output_devices.py:1533: PWMSoftwareFallback: To reduce servo jitter, use the pigpio pin factory.See https://gpiozero.readthedocs.io/en/stable/api_output.html#servo for more info
  'To reduce servo jitter, use the pigpio pin factory.'


I added some code here that is called to create the overlay - basically the box around the face and the score above. You can customize it and/or add information you want to overlay on the camera feed. **This is not needed **

In [5]:
def svg_overlay(faces, frame_size):

    JOY_COLOR = (255, 70, 0)
    SAD_COLOR = (0, 0, 64)

    width, height = frame_size
    doc = svg.Svg(width=width, height=height)

    for face in faces:
        x, y, w, h = face.bounding_box
        fcol='rgb'+str(Color.blend(JOY_COLOR, SAD_COLOR, face.joy_score))
        doc.add(svg.Rect(x=int(x), y=int(y), width=int(w), height=int(h), rx=10, ry=10,
                         fill_opacity=0.3 * face.face_score,
                         style='fill:'+fcol+';stroke:white;stroke-width:4px'))

        doc.add(svg.Text('Joy: %.2f' % face.joy_score, x=x, y=y-10, fill='red', font_size=30))

    return str(doc)

#### Main loop

Here is our main loop based on the code we used last time. Look at the comments on what was changed. Once we see a face, we call the routine `pointFace` which returns the offset of the face boudary box from the center. We also added the streaming back in, to while this cell runs, you can connect to http://orcspi-vis.local:4664 and see the stream.

In [6]:
def pointFace(box, width):
    
    x, y, w, h = box
    middle = x + w/2
    offset = 2*middle/width - 1

    print ("Offset: {offset:.2f}, Middle: {middle}".format(offset=offset, middle=middle))
    
    return -offset

And the main loop we already know so well :)

In [12]:
with contextlib.ExitStack() as stack:
    leds   = stack.enter_context(Leds())
    camera = stack.enter_context(PiCamera(sensor_mode=4, resolution=(820, 616)))

    # This starts and runs the streaming of the camera
    server = stack.enter_context(StreamingServer(camera))  

    print ("Loading model - hold on ..")
    
    # Tune these
    speed = 0.2
    tolerance = 0.2
    duration = 0.2
    
    # Do inference on VisionBonnet
    with CameraInference(face_detection.model()) as inference:
        try:   
            for result in inference.run():
                leds.update(Leds.rgb_on(Color.RED))
                faces = face_detection.get_faces(result)
                
                # This sends the overlay (boxes) to add to the camera stream
                server.send_overlay(svg_overlay(faces, (result.width, result.height)))

                # If we gor a face, check the offset and move
                if len(faces) >= 1:
                    clear_output(wait=True)
                    offset = pointFace(faces[0].bounding_box, result.width)
                    
                    if(offset < -tolerance):
                        bot.right(speed=speed, duration=duration)
                    elif (offset > tolerance):
                        bot.left(speed=speed, duration=duration)
                    
                    leds.update(Leds.rgb_on(Color.GREEN))
                    
        except KeyboardInterrupt:
            print("Interrupted ..")
            
    leds.update(Leds.rgb_off())
    
    # Servo back to the middle upon ending
    bot.stop()
    
    print("Done")

Offset: -0.04, Middle: 790.5
Interrupted ..
Done


[192.168.0.155:53627] Error while processing websocket request
Traceback (most recent call last):
  File "/home/pi/AIY-projects-python/src/aiy/vision/streaming/server.py", line 570, in _receive_message
    packet = self._receive_packet()
  File "/home/pi/AIY-projects-python/src/aiy/vision/streaming/server.py", line 612, in _receive_packet
    packet.fin = buf[0] & 0x80 > 0
IndexError: index out of range
